# Imports

In [1]:
import pandas as pd
import numpy as np
import featuretools as ft
import matplotlib.pyplot as plt

In [2]:
app_train = pd.read_csv(r"C:\Users\HP\Documents\FDS\FDS Project\application_train.csv")

In [3]:
app_test = pd.read_csv(r"C:\Users\HP\Documents\FDS\FDS Project\application_test.csv")

In [4]:
app_train['set'] = 'train'
app_test['set'] = 'test'
app_test["TARGET"] = np.nan

# Append the dataframes
app = app_train.append(app_test, ignore_index = True)

C:\Users\HP\Anaconda3\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [6]:
bureau = pd.read_csv(r"C:\Users\HP\Documents\FDS\FDS Project\bureau.csv")

In [7]:
bureau_balance = pd.read_csv(r"C:\Users\HP\Documents\FDS\FDS Project\bureau_balance.csv")

In [8]:
pos = pd.read_csv(r"C:\Users\HP\Documents\FDS\FDS Project\POS_CASH_balance.csv")

In [9]:
credit = pd.read_csv(r"C:\Users\HP\Documents\FDS\FDS Project\credit_card_balance.csv")

In [10]:
previous = pd.read_csv(r"C:\Users\HP\Documents\FDS\FDS Project\previous_application.csv")

In [11]:
installment = pd.read_csv(r"C:\Users\HP\Documents\FDS\FDS Project\installments_payments.csv")

In [12]:
# Entity set with id applications
es = ft.EntitySet(id = 'clients')

In [13]:
es = es.entity_from_dataframe(entity_id = 'app', dataframe = app, index = 'SK_ID_CURR')

In [14]:
es = es.entity_from_dataframe(entity_id = 'bureau', dataframe = bureau, index = 'SK_ID_BUREAU')

In [15]:
es = es.entity_from_dataframe(entity_id = 'previous', dataframe = previous, index = 'SK_ID_PREV')

In [16]:
es = es.entity_from_dataframe(entity_id = 'bureau_balance', dataframe = bureau_balance, 
                              make_index = True, index = 'bureaubalance_index')

In [17]:
es = es.entity_from_dataframe(entity_id = 'cash', dataframe = pos, 
                              make_index = True, index = 'cash_index')

In [18]:
es = es.entity_from_dataframe(entity_id = 'installments', dataframe = installment,
                              make_index = True, index = 'installments_index')

In [19]:
es = es.entity_from_dataframe(entity_id = 'credit', dataframe = credit,
                              make_index = True, index = 'credit_index')

In [23]:
# Relationship between bureau and bureau balance
r_bureau_balance = ft.Relationship(es['bureau']['SK_ID_BUREAU'], es['bureau_balance']['SK_ID_BUREAU'])

In [24]:
r_app_bureau = ft.Relationship(es['app']['SK_ID_CURR'], es['bureau']['SK_ID_CURR'])
# Relationship between current app and previous apps
r_app_previous = ft.Relationship(es['app']['SK_ID_CURR'], es['previous']['SK_ID_CURR'])

In [25]:
# Relationships between previous apps and cash, installments, and credit
r_previous_cash = ft.Relationship(es['previous']['SK_ID_PREV'], es['cash']['SK_ID_PREV'])
r_previous_installments = ft.Relationship(es['previous']['SK_ID_PREV'], es['installments']['SK_ID_PREV'])
r_previous_credit = ft.Relationship(es['previous']['SK_ID_PREV'], es['credit']['SK_ID_PREV'])

In [26]:
es = es.add_relationships([r_app_bureau, r_bureau_balance, r_app_previous,
                           r_previous_cash, r_previous_installments, r_previous_credit])

In [27]:
import gc
gc.collect()
gc.enable()

In [28]:
feature_matrix, feature_names = ft.dfs(entityset = es, target_entity = 'app',
                                       agg_primitives=["mean"], 
                                        max_depth = 2, features_only=False, verbose = True)

Built 292 features
Elapsed: 06:06 | Progress: 100%|██████████


In [29]:
feature_matrix.to_csv(r"C:\Users\HP\Documents\FDS\FDS Project\featurematrix.csv")